In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType

# Popular Movies 

In [ ]:
spark = SparkSession.builder.appName("Popular Movies").getOrCreate()

schema = StructType([
    StructField("userID", IntegerType(), True),
    StructField("movieID", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True),
])

In [ ]:
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("./resources/datasets/ml-100k/u.data")

In [ ]:
topMoviesID = moviesDF.groupBy("movieID").count().orderBy(func.desc("count"))

In [ ]:
topMoviesID.show()

In [ ]:
spark.stop()

# Movie IDs to Movie Names

In [ ]:
import codecs

In [ ]:
def loadMovieNames():
    moviesNames = {}
    
    with codecs.open("./resources/datasets/ml-100k/u.item", "r", encoding="ISO-8859-1", errors='ignore') as f:
        for line in f:
            print(line)
            fields = line.split('|')
            moviesNames[int(fields[0])] = fields[1]
        return moviesNames

# Superhero dataset

In [3]:
spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

In [4]:
schema = StructType([
                     StructField("id", IntegerType(), True),
                     StructField("name", StringType(), True)])

In [5]:
names = spark.read.schema(schema).option("sep", " ").csv("./resources/datasets/Marvel+Names.txt")

In [6]:
lines = spark.read.text("./resources/datasets/Marvel+Graph.txt")

In [7]:
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

In [8]:
mostPopular = connections.sort(func.col("connections").desc()).first()

In [9]:
mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()

In [10]:
print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")

CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.


Least popular superheroes

In [11]:
minConnects = connections.agg(func.min("connections")).first()[0]

In [12]:
least_connects_superheroes = connections.filter(func.col("connections") == minConnects)

In [13]:
least_superhero_table_with_names = least_connects_superheroes.join(names, "id")

In [14]:
least_superhero_table_with_names.select("names").show()

{"ts": "2025-12-15 21:10:53.285", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `names` cannot be resolved. Did you mean one of the following? [`name`, `id`, `connections`]. SQLSTATE: 42703", "context": {"file": "java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o110.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `names` cannot be resolved. Did you mean one of the following? [`name`, `id`, `connections`]. SQLSTATE: 42703;\n'Project ['names]\n+- Project [id#4, connections#6L, name#1]\n   +- Join Inner, (cast(id#4 as bigint) = cast(id#0 as bigint))\n      :- Filter (connections#6L = cast(0 as bigint)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `names` cannot be resolved. Did you mean one of the following? [`name`, `id`, `connections`]. SQLSTATE: 42703;
'Project ['names]
+- Project [id#4, connections#6L, name#1]
   +- Join Inner, (cast(id#4 as bigint) = cast(id#0 as bigint))
      :- Filter (connections#6L = cast(0 as bigint))
      :  +- Aggregate [id#4], [id#4, sum(connections#5) AS connections#6L]
      :     +- Project [value#2, id#4, (size(split(trim(value#2, None),  , -1), false) - 1) AS connections#5]
      :        +- Project [value#2, split(trim(value#2, None),  , -1)[0] AS id#4]
      :           +- Relation [value#2] text
      +- Relation [id#0,name#1] csv
